# 🌍 M4c: Cross-Lingual Semantic Transfer

## The Big Claim
**"Meaning is universal. Train on English, test on any language."**

## For Paper: "...Until We Found Meaning"
This experiment proves the title is justified.

In [ ]:
!pip install -q sentence-transformers datasets
print("Done")

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import json
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
ENCODER_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'
encoder = SentenceTransformer(ENCODER_NAME)
SEMANTIC_DIM = encoder.get_sentence_embedding_dimension()
print(f"Encoder: {SEMANTIC_DIM}d")

In [ ]:
sst2 = load_dataset('glue', 'sst2')
TRAIN_SIZE = 3000
train_data = sst2['train'].shuffle(seed=42).select(range(TRAIN_SIZE))
test_en = sst2['validation']
print(f"English: train {len(train_data)}, test {len(test_en)}")

In [ ]:
multilingual_tests = {}
lang_names = {'en': 'English', 'de': 'German', 'fr': 'French', 'es': 'Spanish', 'ja': 'Japanese', 'zh': 'Chinese'}

for lang in ['de', 'fr', 'es', 'ja', 'zh']:
    try:
        ds = load_dataset('amazon_reviews_multi', lang, split='test')
        ds_filtered = ds.filter(lambda x: x['stars'] != 3)
        ds_subset = ds_filtered.shuffle(seed=42).select(range(min(500, len(ds_filtered))))
        multilingual_tests[lang] = {
            'texts': [x['review_body'] for x in ds_subset],
            'labels': np.array([1 if x['stars'] >= 4 else 0 for x in ds_subset])}
        print(f"{lang_names[lang]}: {len(ds_subset)}")
    except Exception as e:
        print(f"{lang}: {e}")

In [ ]:
train_texts = list(train_data['sentence'])
train_labels = np.array(train_data['label'])
test_en_texts = list(test_en['sentence'])
test_en_labels = np.array(test_en['label'])

train_emb = encoder.encode(train_texts, show_progress_bar=True)
test_en_emb = encoder.encode(test_en_texts, show_progress_bar=True)

multilingual_emb = {}
for lang, data in multilingual_tests.items():
    multilingual_emb[lang] = encoder.encode(data['texts'], show_progress_bar=False)
    print(f"{lang}: done")

In [ ]:
HDC_DIM = 4096
np.random.seed(42)
projection = np.random.randn(SEMANTIC_DIM, HDC_DIM).astype(np.float32)
projection /= np.linalg.norm(projection, axis=0, keepdims=True)

def to_hdc(emb):
    hdc = emb @ projection
    thr = 0.3 * np.std(hdc, axis=1, keepdims=True)
    return np.where(hdc > thr, 1, np.where(hdc < -thr, -1, 0)).astype(np.float32)

train_hdc = to_hdc(train_emb)
test_en_hdc = to_hdc(test_en_emb)
multilingual_hdc = {lang: to_hdc(emb) for lang, emb in multilingual_emb.items()}
print("HDC done")

In [ ]:
class HDCDataset(Dataset):
    def __init__(self, hdc, labels):
        self.hdc = torch.tensor(hdc, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx): return self.hdc[idx], self.labels[idx]

class HDCClassifier(nn.Module):
    def __init__(self, hdc_dim=4096):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(hdc_dim, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 2))
    def forward(self, x): return self.net(x)

In [ ]:
train_loader = DataLoader(HDCDataset(train_hdc, train_labels), batch_size=32, shuffle=True)
model = HDCClassifier(HDC_DIM).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
criterion = nn.CrossEntropyLoss()

for epoch in range(20):
    model.train()
    for hdc, labels in train_loader:
        hdc, labels = hdc.to(device), labels.to(device)
        optimizer.zero_grad()
        criterion(model(hdc), labels).backward()
        optimizer.step()
    scheduler.step()
    if (epoch+1) % 5 == 0: print(f"Epoch {epoch+1}")
print("Training done")

In [ ]:
def evaluate(hdc, labels):
    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(torch.tensor(hdc).to(device)), dim=1).cpu().numpy()
    return accuracy_score(labels, preds)

results = {}
acc_en = evaluate(test_en_hdc, test_en_labels)
results['en'] = acc_en
print(f"English: {acc_en:.1%}")

for lang, hdc in multilingual_hdc.items():
    acc = evaluate(hdc, multilingual_tests[lang]['labels'])
    results[lang] = acc
    print(f"{lang_names[lang]}: {acc:.1%}")

cross_avg = np.mean([results[l] for l in multilingual_hdc.keys()])
print(f"\nCross-lingual avg: {cross_avg:.1%}")
print(f"Transfer ratio: {cross_avg/acc_en:.1%}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
languages = ['en'] + list(multilingual_hdc.keys())
accs = [results[l] for l in languages]
flags = ['EN', 'DE', 'FR', 'ES', 'JA', 'ZH']
colors = ['green'] + ['lightblue']*5
bars = ax.bar(range(len(languages)), accs, color=colors, edgecolor='black')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xticks(range(len(languages)))
ax.set_xticklabels([f"{flags[i]}\n{lang_names[l]}" for i, l in enumerate(languages)])
ax.set_ylabel('Accuracy')
ax.set_title('Cross-Lingual Transfer: Train English, Test All\n"...Until We Found Meaning"')
ax.set_ylim(0.4, 1.0)
for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{acc:.1%}', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig('m4c_crosslingual_results.png', dpi=150)
plt.show()

In [ ]:
output = {'experiment': 'M4c Cross-Lingual', 'paper': '...Until We Found Meaning',
    'encoder': ENCODER_NAME, 'hdc_dim': HDC_DIM, 'train_lang': 'English', 'train_size': TRAIN_SIZE,
    'results': {lang_names.get(k,k): float(v) for k,v in results.items()},
    'english_acc': float(acc_en), 'crosslingual_avg': float(cross_avg),
    'transfer_ratio': float(cross_avg/acc_en), 'timestamp': datetime.now().isoformat()}
with open('m4c_crosslingual_results.json', 'w') as f: json.dump(output, f, indent=2)
print(json.dumps(output, indent=2))

In [ ]:
if cross_avg > 0.75:
    print("\n✅ STRONG transfer! HDC captures MEANING.")
    print("   Title '...Until We Found Meaning' is JUSTIFIED.")
elif cross_avg > 0.65:
    print("\n📈 Good transfer. Semantic features generalize.")
else:
    print("\n⚠️ Limited transfer.")
print(f"\nKey: Train EN → {cross_avg:.1%} on 5 languages")

In [ ]:
from google.colab import files
files.download('m4c_crosslingual_results.json')
files.download('m4c_crosslingual_results.png')